In [110]:
import pandas
START_DATE = '2011-01-01'
END_DATE = '2017-01-01'
def preprocess(filename, start_date, end_date):
    TradingDaysPeriods = [1, 2, 3, 5, 10, 20, 40, 60, 120, 250]
    
    data = pandas.read_csv(filename, delimiter=',')
    data['Date'] = pd.to_datetime(data['Date'])
    print(data)

    return_features = pandas.DataFrame()
    volume_features = []
    for period in TradingDaysPeriods:
        column = data['Adj Close'].pct_change(periods = period)
        column.rename("Return_" + str(period), inplace = True)
        return_features.append(column)
        
        column = data['Volume'].rolling(period).mean()
        column.rename("AverageVolume_" + str(period), inplace = True)
        volume_features.append(column)
    #return_data = pandas.concat(return_features, axis=1, keys=[s.name for s in return_features])
    return_data = return_features
    volume_data = pandas.concat(volume_features, axis=1, keys=[s.name for s in return_features])
    out_data = pandas.concat([return_data, volume_data], axis=1)
#    for column in volume_features:
#        df.insert(len(df.columns), s.name,s)
    #df = pandas.DataFrame(df, df2)
    #df['Date'] = data['Date']
    out_data.insert(0, 'Date', data['Date'])
    out_data['DayOfWeek'] = out_data['Date'].dt.dayofweek
    out_data['DayOfMonth'] = out_data['Date'].dt.day
    #https://stackoverflow.com/questions/29370057/select-dataframe-rows-between-two-dates
    mask = (out_data['Date'] >= start_date) & (out_data['Date'] <= end_date)
    print(out_data.loc[mask])
    out_data = out_data.loc[mask]
    print(out_data)
    out_data.to_csv(path_or_buf = "features_" + filename) #, sep=';'
    
preprocess('SPY.csv', START_DATE, END_DATE)

           Date        Open        High         Low       Close   Adj Close  \
0    2009-12-31  112.769997  112.800003  111.389999  111.440002   91.689629   
1    2010-01-04  112.370003  113.389999  111.510002  113.330002   93.244675   
2    2010-01-05  113.260002  113.680000  112.849998  113.629997   93.491486   
3    2010-01-06  113.519997  113.989998  113.430000  113.709999   93.557304   
4    2010-01-07  113.500000  114.330002  113.180000  114.190002   93.952278   
...         ...         ...         ...         ...         ...         ...   
1758 2016-12-23  225.429993  225.720001  225.210007  225.710007  214.288986   
1759 2016-12-27  226.020004  226.729996  226.000000  226.270004  214.820618   
1760 2016-12-28  226.570007  226.589996  224.270004  224.399994  213.045227   
1761 2016-12-29  224.479996  224.889999  223.839996  224.350006  212.997772   
1762 2016-12-30  224.729996  224.830002  222.729996  223.529999  212.219269   

         Volume  
0      90637900  
1     118944600

ValueError: All objects passed were None